<a href="https://colab.research.google.com/github/ianaroung/NeuralFCA_Gagarina/blob/main/NeuralFCA_Big_hw_Gagarina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fcapy[all]
!pip install frozendict
!pip install ipynb
!pip install sparselinear
!pip install bitsets
!pip install bitarray
import torch
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cuda118.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cuda118.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cuda118.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.9/162.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.8/118.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.9/279.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.3 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=2a1bcb81848aa0c5d4aa88a25096f9cdb8842801e1a51108371b1a86fefd73f5
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


Looking in links: https://data.pyg.org/whl/torch-2.0.0+cuda118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=495091 sha256=f987854c6d807747692f5897bba71942d5e93e39da2bcad997b8f002803c2607
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cuda118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=1035675 sha256=f91c6d54543b1d2dfb7fe0f594d46176b78c6c72e3ecad2df447703fa54303dc
  Stored in directory: /root/.cache/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
Successfully buil

# Import libs

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from fcapy.context import FormalContext
from fcapy.lattice import ConceptLattice

In [ ]:
from fcapy.visualizer import LineVizNx
import matplotlib.pyplot as plt

plt.rcParams['figure.facecolor'] = (1,1,1,1)

In [ ]:
!python neural_lib.py install

In [ ]:
import neural_lib as nl

# Step 0. Binarize the data

Reading the datasets:

In [ ]:
data1 = pd.read_csv('hcvdat0.csv')
data2 = pd.read_csv('heart_failure_clinical_records_dataset.csv')
data3 = pd.read_excel('Mesothelioma data set.xlsx')

Drop missing values and preprocess some features.

In [ ]:
data1 = data1.iloc[:,1:]
data1 = data1.dropna()
data1 = pd.get_dummies(data1, columns = ['Sex'])
data1['Category'].replace(to_replace=['0=Blood Donor', '0s=suspect Blood Donor', '1=Hepatitis', '2=Fibrosis', '3=Cirrhosis'], value=[0, 0, 1, 2, 3], inplace = True)

data2 = data2.dropna()

data3 = data3.dropna()
#Заменим в target 2 на 1
data3.loc[data3['class of diagnosis'] == 2, 'class of diagnosis'] = 0

The first dataset, whose original target has 4 categories, one of which is healthy people (blood donors), and the other 3 categories are people with different stages of hepatitis C (initial stage, fibrosis and cirrhosis). Thus, we can replace it with 2 categories: people with hepatitis and healthy people.

In [ ]:
data1.loc[data1['Category'] == 0, 'Category'] = 0
data1.loc[data1['Category'] != 0, 'Category'] = 1

The hepatitis C may be indicated by elevated levels of ALT, AST, bilirubin levels, GGT and sometimes creatinine. This information was found on the Internet but we can see it using the next code:

In [ ]:
data1.groupby(['Category']).mean()

So, for the df_bin1 we will use these features:

In [ ]:
df_bin1 = data1[['ALT', 'AST', 'CREA', 'BIL', 'GGT', 'Category']]

In [ ]:
df_bin1.loc[df_bin1['BIL'] >= 10, 'BIL'] = True
df_bin1.loc[df_bin1['BIL'] < 10, 'BIL'] = False
df_bin1.loc[df_bin1['ALT'] < 20, 'ALT'] = True
df_bin1.loc[df_bin1['ALT'] >= 20, 'ALT'] = False
df_bin1.loc[df_bin1['CREA'] < 100, 'CREA'] = True
df_bin1.loc[df_bin1['CREA'] >= 100, 'CREA'] = False
df_bin1.loc[df_bin1['AST'] < 50, 'AST'] = True
df_bin1.loc[df_bin1['AST'] >= 50, 'AST'] = False
df_bin1.loc[df_bin1['GGT'] >= 50, 'GGT'] = True
df_bin1.loc[df_bin1['GGT'] < 50, 'GGT'] = False
df_bin1 = df_bin1.replace({1: True, 0: False})
df_bin1


For this dataset I leave two signs, since according to the study they are the most informative (https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-020-1023-5#Sec8). I also tried to add other features, but they were not important, the neurons mainly used these features, so I decided to split these features into more intervals and remove others so as not to unnecessarily complicate the model.

In [ ]:
data2.groupby(['DEATH_EVENT']).mean()

In [ ]:
df_bin2 = data2[['serum_creatinine', 'ejection_fraction', 'DEATH_EVENT']]

In [ ]:
df_bin2['ejection_fraction>20'] = df_bin2['ejection_fraction']
df_bin2.loc[df_bin2['ejection_fraction>20'] < 20, 'ejection_fraction>20'] = False
df_bin2.loc[df_bin2['ejection_fraction>20'] >= 20, 'ejection_fraction>20'] = True

df_bin2['ejection_fraction>30'] = df_bin2['ejection_fraction']
df_bin2.loc[df_bin2['ejection_fraction>30'] < 30, 'ejection_fraction>30'] = False
df_bin2.loc[df_bin2['ejection_fraction>30'] >= 30, 'ejection_fraction>30'] = True

df_bin2['ejection_fraction>38'] = df_bin2['ejection_fraction']
df_bin2.loc[df_bin2['ejection_fraction>38'] < 38, 'ejection_fraction>38'] = False
df_bin2.loc[df_bin2['ejection_fraction>38'] >= 38, 'ejection_fraction>38'] = True

df_bin2['ejection_fraction>45'] = df_bin2['ejection_fraction']
df_bin2.loc[df_bin2['ejection_fraction>45'] < 45, 'ejection_fraction>45'] = False
df_bin2.loc[df_bin2['ejection_fraction>45'] >= 45, 'ejection_fraction>45'] = True

df_bin2['ejection_fraction>50'] = df_bin2['ejection_fraction']
df_bin2.loc[df_bin2['ejection_fraction>50'] < 50, 'ejection_fraction>50'] = False
df_bin2.loc[df_bin2['ejection_fraction>50'] >= 50, 'ejection_fraction>50'] = True

df_bin2['ejection_fraction>55'] = df_bin2['ejection_fraction']
df_bin2.loc[df_bin2['ejection_fraction>55'] < 55, 'ejection_fraction>55'] = False
df_bin2.loc[df_bin2['ejection_fraction>55'] >= 55, 'ejection_fraction>55'] = True

df_bin2['ejection_fraction>60'] = df_bin2['ejection_fraction']
df_bin2.loc[df_bin2['ejection_fraction>60'] < 60, 'ejection_fraction>60'] = False
df_bin2.loc[df_bin2['ejection_fraction>60'] >= 60, 'ejection_fraction>60'] = True

df_bin2['ejection_fraction>70'] = df_bin2['ejection_fraction']
df_bin2.loc[df_bin2['ejection_fraction>70'] < 70, 'ejection_fraction>70'] = False
df_bin2.loc[df_bin2['ejection_fraction>70'] >= 70, 'ejection_fraction>70'] = True



df_bin2['serum_creatinine<0.9'] = df_bin2['serum_creatinine']
df_bin2.loc[df_bin2['serum_creatinine<0.9'] <= 0.9, 'serum_creatinine<0.9'] = True
df_bin2.loc[df_bin2['serum_creatinine<0.9'] > 0.9, 'serum_creatinine<0.9'] = False

df_bin2['serum_creatinine<1.1'] = df_bin2['serum_creatinine']
df_bin2.loc[df_bin2['serum_creatinine<1.1'] <= 1.1, 'serum_creatinine<1.1'] = True
df_bin2.loc[df_bin2['serum_creatinine<1.1'] > 1.1, 'serum_creatinine<1.1'] = False

df_bin2['serum_creatinine<1.25'] = df_bin2['serum_creatinine']
df_bin2.loc[df_bin2['serum_creatinine<1.25'] <= 1.25, 'serum_creatinine<1.25'] = True
df_bin2.loc[df_bin2['serum_creatinine<1.25'] > 1.25, 'serum_creatinine<1.25'] = False

df_bin2['serum_creatinine<1.4'] = df_bin2['serum_creatinine']
df_bin2.loc[df_bin2['serum_creatinine<1.4'] <= 1.4, 'serum_creatinine<1.4'] = True
df_bin2.loc[df_bin2['serum_creatinine<1.4'] > 1.4, 'serum_creatinine<1.4'] = False

df_bin2['serum_creatinine<1.6'] = df_bin2['serum_creatinine']
df_bin2.loc[df_bin2['serum_creatinine<1.6'] <= 1.6, 'serum_creatinine<1.6'] = True
df_bin2.loc[df_bin2['serum_creatinine<1.6'] > 1.6, 'serum_creatinine<1.6'] = False

df_bin2['serum_creatinine<2'] = df_bin2['serum_creatinine']
df_bin2.loc[df_bin2['serum_creatinine<2'] <= 2, 'serum_creatinine<2'] = True
df_bin2.loc[df_bin2['serum_creatinine<2'] > 2, 'serum_creatinine<2'] = False

df_bin2['serum_creatinine<5'] = df_bin2['serum_creatinine']
df_bin2.loc[df_bin2['serum_creatinine<5'] <= 5, 'serum_creatinine<5'] = True
df_bin2.loc[df_bin2['serum_creatinine<5'] > 5, 'serum_creatinine<5'] = False

df_bin2 = df_bin2.drop(['serum_creatinine'], axis=1)
df_bin2 = df_bin2.drop(['ejection_fraction'], axis=1)

df_bin2 = df_bin2.replace({1: True, 0: False})
df_bin2


There are quite a lot of features in the third dataset - 34, so you can just take all the binary ones, there are 13 of them.

In [ ]:
df_bin3 = data3[['gender', 'asbestos exposure', 'diagnosis method', 'cytology', 'dyspnoea', 'ache on chest',
                'weakness', 'performance status', 'hemoglobin (HGB)', 'dead or not', 'pleural effusion', 'pleural thickness on tomography',
                'pleural level of acidity (pH)', 'class of diagnosis']]

In [ ]:
df_bin3 = df_bin3.replace({1: True, 0: False})
df_bin3

Split the data to train and test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
s = np.arange(0, 700)
a = []
for i in range(len(s)):
  a.append(str(s[i]))

In [ ]:
y1 = df_bin1['Category']
x1 = df_bin1.drop(['Category'], axis=1)

In [ ]:
x1['id'] = a[:589]
x1 = x1.set_index('id')

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.4, random_state=42)

In [ ]:
y2 = df_bin2['DEATH_EVENT']
x2 = df_bin2.drop(['DEATH_EVENT'], axis=1)

In [ ]:
x2['id'] = a[:299]
x2 = x2.set_index('id')

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.4, random_state=42)

In [ ]:
y3 = df_bin3['class of diagnosis']
x3 = df_bin3.drop(['class of diagnosis'], axis=1)

In [ ]:
x3['id'] = a[:324]
x3 = x3.set_index('id')

In [ ]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(x3, y3, test_size=0.4, random_state=42)

## Step 1. Build Monotone Concept Lattice

Put binarized data in FormalContext and compute monotone ConceptLattice

In [ ]:
K_train1 = FormalContext.from_pandas(X_train1)
K_train1

In [ ]:
K_train2 = FormalContext.from_pandas(X_train2)
K_train2

In [ ]:
K_train3 = FormalContext.from_pandas(X_train3)
K_train3

In [ ]:
L1 = ConceptLattice.from_context(K_train1, is_monotone=True)
len(L1)

In [ ]:
L2 = ConceptLattice.from_context(K_train2, is_monotone=True)
len(L2)

In [ ]:
L3 = ConceptLattice.from_context(K_train3, is_monotone=True)
len(L3)

Compute F1 score for each formal concept  (assuming that an object is predicted True if it is in the extent of the concept)

In [ ]:
from sklearn.metrics import f1_score

for c1 in L1:
    y_preds1 = np.zeros(K_train1.n_objects)
    y_preds1[list(c1.extent_i)] = 1
    c1.measures['f1_score'] = f1_score(y_train1, y_preds1)

In [ ]:
from sklearn.metrics import f1_score

for c2 in L2:
    y_preds2 = np.zeros(K_train2.n_objects)
    y_preds2[list(c2.extent_i)] = 1
    c2.measures['f1_score'] = f1_score(y_train2, y_preds2)

In [ ]:
from sklearn.metrics import f1_score

for c3 in L3:
    y_preds3 = np.zeros(K_train3.n_objects)
    y_preds3[list(c3.extent_i)] = 1
    c3.measures['f1_score'] = f1_score(y_train3, y_preds3)

Select indices of the best concepts from the lattice

In [ ]:
best_concepts1 = list(L1.measures['f1_score'].argsort()[::-1][3:4])

# assert len({g_i for c in L1[best_concepts1] for g_i in c.extent_i})==K_train1.n_objects, "Selected concepts do not cover all train objects"

In [ ]:
best_concepts2 = list(L2.measures['f1_score'].argsort()[::-1][:2])

assert len({g_i for c in L2[best_concepts2] for g_i in c.extent_i})==K_train2.n_objects, "Selected concepts do not cover all train objects"

In [ ]:
best_concepts3 = list(L3.measures['f1_score'].argsort()[::-1][:194])

assert len({g_i for c in L3[best_concepts3] for g_i in c.extent_i})==K_train3.n_objects, "Selected concepts do not cover all train objects"

Construct neural network based on concept lattice

In [ ]:
cn1 = nl.ConceptNetwork.from_lattice(L1, best_concepts1, sorted(set(y_train1)))

In [ ]:
cn2 = nl.ConceptNetwork.from_lattice(L2, best_concepts2, sorted(set(y_train2)))

In [ ]:
cn3 = nl.ConceptNetwork.from_lattice(L3, best_concepts3, sorted(set(y_train3)))

Setup visuzalier for the architecture

In [ ]:
vis1 = LineVizNx(node_label_font_size=14, node_label_func=lambda el_i, P: nl.zneuron_label_func(el_i, P, set(cn1.attributes))+'\n\n')

In [ ]:
vis2 = LineVizNx(node_label_font_size=14, node_label_func=lambda el_i, P: nl.zneuron_label_func(el_i, P, set(cn2.attributes))+'\n\n')

In [ ]:
vis3 = LineVizNx(node_label_font_size=14, node_label_func=lambda el_i, P: nl.zneuron_label_func(el_i, P, set(cn3.attributes))+'\n\n')

In [ ]:
descr1 = {'BIL', 'ALT'}

traced1 = cn1.trace_description(descr1, include_targets=False)

In [ ]:
descr2 = {'serum_creatinine<1.4', 'ejection_fraction>30'}

traced2 = cn2.trace_description(descr2, include_targets=False)

In [ ]:
descr3 = {'dead or not'}

traced3 = cn3.trace_description(descr3, include_targets=False)

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

vis1.draw_poset(
    cn1.poset, ax=ax,
    flg_node_indices=False,
    node_label_func=lambda el_i, P: nl.neuron_label_func(el_i, P, set(cn1.attributes), only_new_attrs=True)+'\n\n',
    node_color=['darkblue' if el_i in traced1 else 'lightgray' for el_i in range(len(cn1.poset))]
)
plt.title(f'NN based on 353 best concepts from monotone concept lattice', loc='left', x=0.05, size=24)

plt.text(max(vis1.mover.posx), min(vis1.mover.posy)-0.3, f'*Blue neurons are the ones activated by description {descr1}', fontsize=14, ha='right', color='dimgray')

plt.subplots_adjust()
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

vis2.draw_poset(
    cn2.poset, ax=ax,
    flg_node_indices=False,
    node_label_func=lambda el_i, P: nl.neuron_label_func(el_i, P, set(cn2.attributes), only_new_attrs=True)+'\n\n',
    node_color=['darkblue' if el_i in traced2 else 'lightgray' for el_i in range(len(cn2.poset))]
)
plt.title(f'NN based on 180 best concepts from monotone concept lattice', loc='left', x=0.05, size=24)

plt.text(max(vis2.mover.posx), min(vis2.mover.posy)-0.3, f'*Blue neurons are the ones activated by description {descr2}', fontsize=14, ha='right', color='dimgray')

plt.subplots_adjust()
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

vis3.draw_poset(
    cn3.poset, ax=ax,
    flg_node_indices=False,
    node_label_func=lambda el_i, P: nl.neuron_label_func(el_i, P, set(cn3.attributes), only_new_attrs=True)+'\n\n',
    node_color=['darkblue' if el_i in traced3 else 'lightgray' for el_i in range(len(cn3.poset))]
)
plt.title(f'NN based on 194 best concepts from monotone concept lattice', loc='left', x=0.05, size=24)

plt.text(max(vis3.mover.posx), min(vis3.mover.posy)-0.3, f'*Blue neurons are the ones activated by description {descr3}', fontsize=14, ha='right', color='dimgray')

plt.subplots_adjust()
plt.tight_layout()
plt.show()

Now fit the network

In [ ]:
cn1.fit(X_train1, y_train1)

In [ ]:
cn2.fit(X_train2, y_train2)

In [ ]:
cn3.fit(X_train3, y_train3)

To obtain the prediction

In [ ]:
print('Class prediction', cn1.predict(X_test1).numpy())
print('Class prediction with probabilities', cn1.predict_proba(X_test1).detach().numpy())
print('True class', y_test1.values)

In [ ]:
(cn1.predict(X_test1).numpy() == y_test1).sum()/len(y_test1)

In [ ]:
f1_score(y_test1, cn1.predict(X_test1).numpy())

In [ ]:
print('Class prediction', cn2.predict(X_test2).numpy())
print('Class prediction with probabilities', cn2.predict_proba(X_test2).detach().numpy())
print('True class', y_test2.values)

In [ ]:
f1_score(y_test2, cn2.predict(X_test2).numpy())

In [ ]:
(cn2.predict(X_test2).numpy() == y_test2).sum()/len(y_test2)

In [ ]:
print('Class prediction', cn3.predict(X_test3).numpy())
print('Class prediction with probabilities', cn3.predict_proba(X_test3).detach().numpy())
print('True class', y_test3.values)

In [ ]:
f1_score(y_test3, cn3.predict(X_test3).numpy())

In [ ]:
(cn3.predict(X_test3).numpy() == y_test3).sum()/len(y_test3)

We see that the model predicts perfect the third dataset.

Let us look at the fitted weights of edges

In [ ]:
edge_weights1 = cn1.edge_weights_from_network()

In [ ]:
edge_weights2 = cn2.edge_weights_from_network()

In [ ]:
edge_weights3 = cn3.edge_weights_from_network()

In [ ]:
import networkx as nx

fig, ax = plt.subplots(figsize=(15,5))

vis1.draw_poset(
    cn1.poset, ax=ax,
    flg_node_indices=False,
    node_label_func=lambda el_i, P: nl.neuron_label_func(el_i, P, set(cn1.attributes), only_new_attrs=True)+'\n\n',
    edge_color=[edge_weights1[edge] for edge in cn1.poset.to_networkx().edges],
    edge_cmap=plt.cm.RdBu,
)
nx.draw_networkx_edge_labels(cn1.poset.to_networkx(), vis1.mover.pos, {k: f"{v:.1f}" for k,v in edge_weights1.items()}, label_pos=0.7)

plt.title('Neural network with fitted edge weights', size=24, x=0.05, loc='left')
plt.tight_layout()
plt.subplots_adjust()
plt.savefig('fitted_network.png')
plt.show()

In [ ]:
import networkx as nx

fig, ax = plt.subplots(figsize=(15,5))

vis2.draw_poset(
    cn2.poset, ax=ax,
    flg_node_indices=False,
    node_label_func=lambda el_i, P: nl.neuron_label_func(el_i, P, set(cn2.attributes), only_new_attrs=True)+'\n\n',
    edge_color=[edge_weights2[edge] for edge in cn2.poset.to_networkx().edges],
    edge_cmap=plt.cm.RdBu,
)
nx.draw_networkx_edge_labels(cn2.poset.to_networkx(), vis2.mover.pos, {k: f"{v:.1f}" for k,v in edge_weights2.items()}, label_pos=0.7)

plt.title('Neural network with fitted edge weights', size=24, x=0.05, loc='left')
plt.tight_layout()
plt.subplots_adjust()
plt.savefig('fitted_network.png')
plt.show()

In [ ]:
import networkx as nx

fig, ax = plt.subplots(figsize=(15,5))

vis3.draw_poset(
    cn3.poset, ax=ax,
    flg_node_indices=False,
    node_label_func=lambda el_i, P: nl.neuron_label_func(el_i, P, set(cn3.attributes), only_new_attrs=True)+'\n\n',
    edge_color=[edge_weights3[edge] for edge in cn3.poset.to_networkx().edges],
    edge_cmap=plt.cm.RdBu,
)
nx.draw_networkx_edge_labels(cn3.poset.to_networkx(), vis3.mover.pos, {k: f"{v:.1f}" for k,v in edge_weights3.items()}, label_pos=0.7)

plt.title('Neural network with fitted edge weights', size=24, x=0.05, loc='left')
plt.tight_layout()
plt.subplots_adjust()
plt.savefig('fitted_network.png')
plt.show()

It is a good model but because of a lot of attributes the graph does not look pretty.